# Parallel DDM Simulators Benchmark

This notebook compares the performance of different DDM (Drift Diffusion Model) simulation backends:

| Backend | Description | Parallelism |
|---------|-------------|-------------|
| **Cython Original** | The original ssm-simulators implementation | Sequential |
| **Rust (Single-core)** | Rust with PCG64 + Ziggurat RNG | Sequential |
| **Rust (Parallel)** | Rust with Rayon work-stealing parallelism | Multi-threaded |

All implementations generate **fresh random numbers** throughout (statistically correct).

In [ ]:
import numpy as np
import time
import os
import matplotlib.pyplot as plt

# Set plot style
plt.style.use("seaborn-v0_8-whitegrid")
plt.rcParams["figure.figsize"] = (12, 6)
plt.rcParams["font.size"] = 12

print(f"Available CPU cores: {os.cpu_count()}")

## 1. Import Simulators

In [ ]:
# Original Cython implementation
from cssm.ddm_models import ddm as ddm_cython

# Rust implementations (optimized with PCG64 + Ziggurat RNG)
from ssms.parallel_backends.rust_parallel import (
    ddm_rust,  # Parallel (uses all cores)
    ddm_rust_single,  # Single-threaded
    get_rust_info,
)

# Show Rust backend info
rust_info = get_rust_info()
print(f"Rust backend: {rust_info}")

## 2. Setup Parameters

We'll create random DDM parameters for multiple trials to test realistic workloads.

In [ ]:
def create_params(n_trials: int, seed: int = 42) -> dict:
    """Create random DDM parameters for n_trials."""
    np.random.seed(seed)
    return {
        "v": np.random.uniform(0.3, 0.8, n_trials).astype(np.float32),  # drift rate
        "a": np.random.uniform(1.0, 2.0, n_trials).astype(np.float32),  # boundary
        "z": np.random.uniform(0.4, 0.6, n_trials).astype(
            np.float32
        ),  # starting point (relative)
        "t": np.random.uniform(0.2, 0.4, n_trials).astype(
            np.float32
        ),  # non-decision time
        "deadline": np.full(n_trials, 999.0, dtype=np.float32),  # deadline
        "s": np.ones(n_trials, dtype=np.float32),  # noise std
    }


# Quick verification
params_test = create_params(5)
print("Sample parameters (5 trials):")
for k, v in params_test.items():
    print(f"  {k}: {v}")

## 3. Verify Correctness

Before benchmarking, let's verify all implementations produce similar output distributions.

In [ ]:
# Small test case
n_trials = 10
n_samples = 10000
params = create_params(n_trials)

# Run all backends
result_cython = ddm_cython(
    **params,
    n_samples=n_samples,
    n_trials=n_trials,
    random_state=42,
    return_option="minimal",
)
result_rust_single = ddm_rust_single(
    **params, n_samples=n_samples, n_trials=n_trials, random_state=42
)
result_rust_parallel = ddm_rust(
    **params, n_samples=n_samples, n_trials=n_trials, random_state=42
)

# Compare statistics
print("Output Statistics (mean RT, choice proportion):")
print("=" * 60)

for name, result in [
    ("Cython", result_cython),
    ("Rust Single", result_rust_single),
    ("Rust Parallel", result_rust_parallel),
]:
    rts = result[0].flatten()
    choices = result[1].flatten()

    mean_rt = np.mean(rts)
    std_rt = np.std(rts)
    prop_upper = np.mean(choices == 1)

    print(f"{name:15s}: RT = {mean_rt:.3f} ± {std_rt:.3f}, P(upper) = {prop_upper:.3f}")

In [ ]:
# Visualize RT distributions
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

results = [
    ("Cython Original", result_cython, "#2ecc71"),
    ("Rust Single", result_rust_single, "#e74c3c"),
    ("Rust Parallel", result_rust_parallel, "#3498db"),
]

for ax, (name, result, color) in zip(axes, results):
    rts = result[0].flatten()
    choices = result[1].flatten()

    # Plot upper and lower boundary RTs
    ax.hist(
        rts[choices == 1], bins=50, alpha=0.7, label="Upper", color=color, density=True
    )
    ax.hist(
        -rts[choices == -1],
        bins=50,
        alpha=0.7,
        label="Lower",
        color="gray",
        density=True,
        hatch="//",
    )

    ax.set_xlabel("RT (s)")
    ax.set_ylabel("Density")
    ax.set_title(name)
    ax.legend()
    ax.set_xlim(-3, 3)

plt.tight_layout()
plt.suptitle("RT Distributions (should be similar across backends)", y=1.02)
plt.show()

## 4. Performance Benchmark

Now let's benchmark at various scales.

In [ ]:
def benchmark_backend(func, params, n_samples, n_trials, n_runs=3, **kwargs):
    """Benchmark a backend and return mean time."""
    times = []
    for _ in range(n_runs):
        start = time.perf_counter()
        _ = func(**params, n_samples=n_samples, n_trials=n_trials, **kwargs)
        times.append(time.perf_counter() - start)
    return np.mean(times), np.std(times)


# Warmup
print("Warming up backends...")
small_params = create_params(10)
_ = ddm_cython(
    **small_params, n_samples=100, n_trials=10, random_state=42, return_option="minimal"
)
_ = ddm_rust(**small_params, n_samples=100, n_trials=10, random_state=42)
print("Done!")

In [ ]:
# Define benchmark configurations
configs = [
    (100, 1000),  # 100K simulations (small)
    (100, 10000),  # 1M simulations (medium)
    (100, 50000),  # 5M simulations (large)
    (100, 100000),  # 10M simulations (very large)
]

backends = [
    ("Cython Original", ddm_cython, {"random_state": 42, "return_option": "minimal"}),
    ("Rust (1 core)", ddm_rust_single, {"random_state": 42}),
    ("Rust Parallel", ddm_rust, {"random_state": 42}),
]

# Run benchmarks
benchmark_results = {name: [] for name, _, _ in backends}
total_sims = []

print("Running benchmarks...")
print("=" * 70)

for n_trials, n_samples in configs:
    total = n_trials * n_samples
    total_sims.append(total)
    params = create_params(n_trials)

    print(f"\n{total:,} simulations ({n_trials} trials × {n_samples} samples):")

    for name, func, kwargs in backends:
        mean_time, std_time = benchmark_backend(
            func, params, n_samples, n_trials, **kwargs
        )
        benchmark_results[name].append(mean_time)
        throughput = total / mean_time
        print(
            f"  {name:20s}: {mean_time:6.2f}s ± {std_time:.2f}s  ({throughput / 1e6:.1f}M sim/s)"
        )

print("\nDone!")

In [ ]:
# Visualize benchmark results
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

colors = {
    "Cython Original": "#2ecc71",
    "Rust (1 core)": "#e74c3c",
    "Rust Parallel": "#3498db",
}

# Plot 1: Absolute time
ax = axes[0]
x = np.arange(len(configs))
width = 0.25

for i, (name, times) in enumerate(benchmark_results.items()):
    ax.bar(x + i * width, times, width, label=name, color=colors[name], alpha=0.8)

ax.set_xlabel("Total Simulations")
ax.set_ylabel("Time (seconds)")
ax.set_title("Execution Time by Backend")
ax.set_xticks(x + width)
ax.set_xticklabels([f"{t / 1e6:.1f}M" for t in total_sims])
ax.legend()
ax.set_yscale("log")

# Plot 2: Speedup vs Cython
ax = axes[1]
baseline = np.array(benchmark_results["Cython Original"])

for name, times in benchmark_results.items():
    speedup = baseline / np.array(times)
    ax.plot(
        total_sims,
        speedup,
        "o-",
        label=name,
        color=colors[name],
        linewidth=2,
        markersize=8,
    )

ax.axhline(y=1, color="gray", linestyle="--", alpha=0.5)
ax.set_xlabel("Total Simulations")
ax.set_ylabel("Speedup vs Cython Original")
ax.set_title("Speedup Factor")
ax.set_xscale("log")
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 5. Summary Table

In [ ]:
# Create summary table for the largest benchmark
largest_idx = -1
baseline_time = benchmark_results["Cython Original"][largest_idx]
total = total_sims[largest_idx]

print(f"\n{'=' * 70}")
print(f"SUMMARY: {total / 1e6:.0f}M Simulations Benchmark")
print(f"{'=' * 70}")
print(f"{'Backend':<25} {'Time (s)':<12} {'Throughput':<18} {'Speedup':<10}")
print("-" * 70)

for name in benchmark_results:
    t = benchmark_results[name][largest_idx]
    throughput = total / t
    speedup = baseline_time / t
    print(f"{name:<25} {t:<12.2f} {throughput / 1e6:>14.1f}M/s {speedup:>8.1f}x")

print(f"{'=' * 70}")
print("\nNotes:")
print("  - All backends generate fresh random numbers (statistically correct)")
print("  - Rust uses PCG64 + Ziggurat RNG (same algorithms as NumPy)")
print(f"  - Parallel Rust uses Rayon work-stealing across {os.cpu_count()} cores")

## 6. RNG Algorithm Comparison

The Rust backend uses the same RNG algorithms as NumPy for maximum performance:

| Component | NumPy | Rust |
|-----------|-------|------|
| **PRNG** | PCG64 | PCG64 (via `rand_pcg`) |
| **Normal distribution** | Ziggurat | Ziggurat (via `rand_distr::StandardNormal`) |
| **Speed** | ~240M samples/s | ~240M samples/s |

The key advantage of Rust is that random numbers are generated **inline** during the simulation,
avoiding Python callback overhead that plagues Cython implementations.

In [ ]:
# Demonstrate RNG speed
print("RNG Speed Comparison:")
print("=" * 50)

n = 50_000_000

# NumPy
rng = np.random.default_rng(42)
start = time.perf_counter()
_ = rng.standard_normal(n, dtype=np.float32)
numpy_time = time.perf_counter() - start
print(f"NumPy (PCG64 + Ziggurat): {n / numpy_time / 1e6:.0f}M samples/s")

print("\nBoth NumPy and Rust use the same fast algorithms!")

## 7. Usage Examples

In [ ]:
# Example: Using the Rust parallel backend
from ssms.parallel_backends.rust_parallel import ddm_rust

# Single trial parameters
result = ddm_rust(
    v=np.array([0.5], dtype=np.float32),
    a=np.array([1.5], dtype=np.float32),
    z=np.array([0.5], dtype=np.float32),
    t=np.array([0.3], dtype=np.float32),
    deadline=np.array([10.0], dtype=np.float32),
    s=np.array([1.0], dtype=np.float32),
    n_samples=10000,
    n_trials=1,
    random_state=42,
)

rts, choices = result
print(f"Generated {len(rts.flatten())} samples")
print(f"Mean RT: {np.mean(rts):.3f}s")
print(f"P(upper boundary): {np.mean(choices == 1):.3f}")

In [ ]:
# Example: Multiple trials with different parameters
n_trials = 50
n_samples = 1000

# Different drift rates for each trial
drift_rates = np.linspace(0.1, 1.0, n_trials).astype(np.float32)

result = ddm_rust(
    v=drift_rates,
    a=np.full(n_trials, 1.5, dtype=np.float32),
    z=np.full(n_trials, 0.5, dtype=np.float32),
    t=np.full(n_trials, 0.3, dtype=np.float32),
    deadline=np.full(n_trials, 10.0, dtype=np.float32),
    s=np.ones(n_trials, dtype=np.float32),
    n_samples=n_samples,
    n_trials=n_trials,
    random_state=42,
)

rts, choices = result
print(f"Shape: {rts.shape} (n_samples × n_trials)")

# Plot accuracy vs drift rate
accuracy = np.mean(choices == 1, axis=0)

plt.figure(figsize=(10, 4))
plt.plot(drift_rates, accuracy, "o-", color="#3498db", linewidth=2, markersize=6)
plt.xlabel("Drift Rate (v)")
plt.ylabel("P(upper boundary)")
plt.title("Accuracy increases with drift rate")
plt.grid(True, alpha=0.3)
plt.show()

## Conclusion

The **Rust parallel backend** provides:

1. **~20-25x speedup** over the original Cython implementation
2. **Statistically correct** random number generation (fresh numbers throughout)
3. **Same RNG quality** as NumPy (PCG64 + Ziggurat)
4. **Automatic parallelization** via Rayon work-stealing

For production workloads, especially large-scale parameter recovery or model fitting,
the Rust backend is strongly recommended.